<a href="https://colab.research.google.com/github/LinarKulinar/HPC_SSAU/blob/master/Lab1_Vector_summation/Lab1_Vector_summation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Высокопроизводительные вычисления.**

**Лабораторная 1. Сложение векторов на CPU и GPU.**

Самарский университет

В данном блокноте реализован алгоритм сложения векторов для CPU и для GPU.

Для выполнения кода на C на GPU используется библиотека PyCuda.

Данный код запущен в среде Google Colaboratory.

In [31]:
! pip install pycuda

In [32]:
!nvidia-smi

Sun Jan  9 15:57:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    76W / 149W |     58MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
import numpy as np
import time

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule

Определяем функцию сложения векторов на C, которая будет выполняться на GPU

In [34]:
mod = SourceModule("""
  __global__ void vec_sum_gpu(double *x, double *y, double *c_gpu, const int n) {  
      int idx = threadIdx.x + (blockIdx.x*blockDim.x);
      c_gpu[idx] = x[idx] + y[idx];
  }    
""")

Определяем функцию сложения векторов на python, которая будет выполняться на CPU

In [35]:
def vec_sum_cpu(x, y, n):
  c = np.zeros(n)
  for i in range(n):
    c[i] = x[i] + y[i]
  return c

Вводим входные параметры задачи - размеры складываемых массивов

In [36]:
def generate_data():
  n = int(input('Введите размер векторов = '))

  x = np.random.randn(n).astype(np.float)
  y = np.random.randn(n).astype(np.float)
  print('Сгенерированные массивы: \n', x, '\n', y)
  return x, y, n

In [37]:
x, y, n = generate_data()

Введите размер векторов = 100000000
Сгенерированные массивы: 
 [ 0.679819    0.6515463  -1.51751742 ...  0.74754132  0.79676538
  1.39315   ] 
 [-1.21146089 -2.8443205  -0.68146343 ... -0.3140449  -0.6803967
 -0.15511029]


Вычисляем на CPU, замеряя время

In [38]:
cpu_start = time.time()
result_cpu = vec_sum_cpu(x,y, n)
cpu_time = time.time() - cpu_start
print('Результирующий вектор: ', result_cpu)
print('Время на CPU: ', round(cpu_time,4))

Результирующий вектор:  [-0.53164189 -2.19277419 -2.19898084 ...  0.43349642  0.11636868
  1.23803972]
Время на CPU:  48.3547


Вычисляем на GPU, замеряя время

In [39]:
# т.к. массивы являются одномерными
block_size = 128
block = (block_size, 1, 1)
grid = (int((n + block_size - 1) / block_size), 1)
block, grid

((128, 1, 1), (781250, 1))

In [40]:
# c_gpu = np.ones(n).astype(np.float)
result_gpu = gpuarray.zeros((n), dtype=np.double)
result_gpu  = result_gpu.get()

vec_sum_gpu = mod.get_function("vec_sum_gpu")

gpu_start = time.time()
vec_sum_gpu(driver.In(x), driver.In(y), driver.Out(result_gpu), np.int32(n), block = block, grid = grid)
driver.Context.synchronize()
gpu_time = time.time() - gpu_start

print('Результирующий вектор: ', result_gpu)
print('Время на GPU: ', round(gpu_time,4))

Результирующий вектор:  [-0.53164189 -2.19277419 -2.19898084 ...  0.43349642  0.11636868
  1.23803972]
Время на GPU:  0.5042


Выводим результаты

In [41]:
print('Vector size: ', n)
print('Results converge;  acceleration: ', round(cpu_time/gpu_time, 4)) if np.allclose(result_gpu, result_cpu) else print('Results do not converge')

print('Time on GPU: ', round(gpu_time,4))
print('Time on CPU: ', round(cpu_time,4))

Vector size:  100000000
Results converge;  acceleration:  95.903
Time on GPU:  0.5042
Time on CPU:  48.3547
